In [4]:
from tfidf import raw_prefix
from puc import models
from keras.models import Model
from keras.layers import Input, Dense, Embedding, Concatenate
from keras.layers import LSTM
import numpy as np
import pandas as pd
from functools import partial, reduce
from sklearn.model_selection import train_test_split

seed = 42
np.random.seed(seed)
rnn_num_samples=100
lstm_size = 128
dropout = 0.2
recurrent_dropout = 0.2
epochs = 20
batch_size = 128 # number of datapoints its fed each time it fits
pad_length_lig = 1000 # max size of smile string
pad_length_seq = 1000 # max size of sequence
num_puc_models = 30
num_bindings = 300

print(len(models.keys()))

list(models.items())[:3]

35


[(1039367,
  BaggingClassifierPU(base_estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, presort=False, random_state=None,
              splitter='best'),
            bootstrap=True, bootstrap_features=False, max_features=1.0,
            max_samples=8, n_estimators=100, n_jobs=1, oob_score=True,
            random_state=None, verbose=0, warm_start=False)),
 (1189571,
  BaggingClassifierPU(base_estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, presort=False, random_state=None,
         

In [ ]:

ligands = (pd.read_csv(raw_prefix+'ligands.csv', 
                       index_col='id', 
                       usecols=['id', 'SMILES'])
           .rename({'SMILES': 'smile'}, axis=1))


sequences = pd.read_csv(raw_prefix+'sequences.csv', index_col=0)#.rename({'Unnamed: 0': 'seq_id'}, axis=1)

ligands['smile_length'] = ligands.smile.apply(lambda x: len(x))
ligands = ligands.loc[ligands.smile_length<=pad_length_lig].loc[ligands.smile_length>0]

sequences['seq_length'] = sequences.sequence.apply(lambda x: len(x))
sequences = sequences.loc[sequences.seq_length<=pad_length_seq].loc[sequences.seq_length>0]

PUC_models_list = np.random.choice([k for k,v 
                                    in models.items() 
                                    if k in ligands.index 
                                    and k in sequences.index], num_puc_models)

lig2seq = pd.read_csv(raw_prefix+'lig2seq.csv').rename({'lig': 'lig_idx', 'seq': 'seq_idx'}, axis=1)

lig2seq = lig2seq.loc[lig2seq.lig_idx.isin(ligands.index)]
lig2seq = lig2seq.loc[lig2seq.seq_idx.isin(sequences.index)]

lig2seq = lig2seq.loc[~lig2seq.lig_idx.isin(PUC_models_list)].sample(n=num_bindings)

print(ligands.shape, sequences.shape, lig2seq.shape)


In [ ]:
import sys
import time
 
def spinning_cursor():
    while True:
        for cursor in '|/-\\':
            yield cursor

spinner = spinning_cursor()

for _ in range(50):
    sys.stdout.write(next(spinner))
    sys.stdout.flush()
    time.sleep(0.1)
    sys.stdout.write('\b')